### Importing Packages

In [1]:
import numpy as np
import pandas as pd
import cv2
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten

Using TensorFlow backend.


### CNN Model

In [2]:
classifier = Sequential()
classifier.add(Convolution2D(64,3,3,input_shape = (64,64,3),activation = 'relu'))
# classifier.add(Convolution2D(64,3,3,activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
# classifier.add(Convolution2D(64,3,3,activation = 'relu'))
classifier.add(Convolution2D(64,3,3,activation = 'relu'))
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(128,activation='relu'))
classifier.add(Dense(128,activation='relu'))
classifier.add(Dense(1,activation='sigmoid'))
classifier.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


C:\Users\spars\miniconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), input_shape=(64, 64, 3..., activation="relu")`
  
C:\Users\spars\miniconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  


### Traing and Testing the data

In [3]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64,64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64,64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         samples_per_epoch = 8000,
                         nb_epoch = 25,
                         validation_data = test_set,
                         nb_val_samples = 2000)


Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.


C:\Users\spars\miniconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
C:\Users\spars\miniconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=<keras_pre..., steps_per_epoch=250, epochs=25, validation_steps=2000)`


Epoch 1/25
250/250 [==============================] - 262s 1s/step - loss: 0.6676 - acc: 0.5917 - val_loss: 0.6031 - val_acc: 0.6782
Epoch 2/25
250/250 [==============================] - 250s 1000ms/step - loss: 0.6042 - acc: 0.6712 - val_loss: 0.5522 - val_acc: 0.7236
Epoch 3/25
250/250 [==============================] - 254s 1s/step - loss: 0.5565 - acc: 0.7124 - val_loss: 0.5204 - val_acc: 0.7500
Epoch 4/25
250/250 [==============================] - 253s 1s/step - loss: 0.5291 - acc: 0.7374 - val_loss: 0.5735 - val_acc: 0.7185
Epoch 5/25
250/250 [==============================] - 253s 1s/step - loss: 0.4978 - acc: 0.7620 - val_loss: 0.4943 - val_acc: 0.7729
Epoch 6/25
250/250 [==============================] - 250s 999ms/step - loss: 0.4723 - acc: 0.7741 - val_loss: 0.4644 - val_acc: 0.7784
Epoch 7/25
250/250 [==============================] - 261s 1s/step - loss: 0.4629 - acc: 0.7731 - val_loss: 0.4553 - val_acc: 0.7885
Epoch 8/25
250/250 [==============================] - 256s 1s/

### We get an Accuracy of 92%, which indicates a Good Model

#### Predicting our own handwritten digit using OpenCV

In [ ]:
import cv2
cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
while True:
    ret, frame = cam.read()
    cv2.imshow("test", frame)
    if ret == False:
        break
    k = cv2.waitKey(1)

    if k%256 == 27:      # ESC pressed
        break
    elif k%256 == 32:    # SPACE pressed
        cv2.imwrite('test_image.jpg', frame)
cam.release()
cv2.destroyAllWindows()


classifier.save('model.h5')
from keras.models import load_model
model = load_model('model.h5')
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

img = cv2.imread('test_image.jpg')
img = cv2.resize(img,(64,64))
img = np.reshape(img,[1,64,64,3])

pred = model.predict_classes(img)

print(pred) 